In [1]:
import pandas as pd
import numpy as np

In [2]:
STREAM_COUNT = 10

In [3]:
n1 = {
    'id':['s'+str(i) for i in range(1,STREAM_COUNT+1)],
    'O' :[None,0   ,0   ,0   ,0   ,0   ,0   ,0   ],
    'OJ':[0   ,0   ,0   ,55.8,0   ,None,0   ,None],
    'PS':[0   ,0   ,0   ,21.3,None,0   ,0   ,0   ],
    'W' :[0   ,None,None,41.5,None,None,None,None]
}

n2 = {
    'Species':['A'  ,'B'  ,'C'  ,'D'  ],
    'S1'     :[None, None, None, None ],
    'S2'     :[None ,None ,0    ,0    ],
    'S3'     :[None ,None ,0    ,0    ],
    'S4'     :[0    ,None ,None ,None ],
    'S5'     :[None ,None ,0    ,0    ],
    'S6'     :[None ,None ,0    ,0    ],
    'S7'     :[None ,None ,0    ,0    ],
    'S8'     :[0    ,None ,None ,0    ],
    'S9'     :[0    ,0    ,None ,None ],
    'S10'    :[None ,None ,None ,0    ]
}

x2 = {
    'Species':['A'  ,'B'  ,'C'  ,'D'  ],
    'S1'     :[0.264,None ,0.212, None],
    'S2'     :[0.578,None ,0    ,0    ],
    'S3'     :[None ,None ,0    ,0    ],
    'S4'     :[0    ,None ,None ,None ],
    'S5'     :[0.905,None ,0    ,0    ],
    'S6'     :[None ,None ,0    ,0    ],
    'S7'     :[None ,None ,0    ,0    ],
    'S8'     :[0    ,0.268,None ,0    ],
    'S9'     :[0    ,0    ,None ,None ],
    'S10'    :[0.108,0.465,None ,0    ]
}

n = pd.DataFrame(n2).set_index('Species')
n.S1 = pd.to_numeric(n.S1)

x = pd.DataFrame(x2).set_index('Species')

### Fill in existing information

In [4]:
def totals_from_components(df):
    sum_if_complete = lambda v: sum(v) if 'NaN' not in v else 'NaN'
    if 'total' not in df.index:
        df.loc['total'] = df.apply(sum_if_complete)
    else:
        for col in df:
            if np.isnan(df[col].total):
                df[col].total = sum_if_complete(df.drop('total')[col])

totals_from_components(n)
n.S10.total = 1000 #mol/min
display(n)
display(x)

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
Species,,,,,,,,,,
A,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN
B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
C,NaN,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN
D,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0
total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
Species,,,,,,,,,,
A,0.264,0.578,NaN,0.0,0.905,NaN,NaN,0.000,0.0,0.108
B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.268,0.0,0.465
C,0.212,0.000,0.0,NaN,0.000,0.0,0.0,NaN,NaN,NaN
D,NaN,0.000,0.0,NaN,0.000,0.0,0.0,0.000,NaN,0.000


In [5]:
def countNaN(array): return len([X for X in array if np.isnan(X)])
def complete_ratios(df):
    for S in df:
        if countNaN(df[S].values) is 1:
            for i in df.index:
                if np.isnan(df[S][i]):
                    df[S][i] = 1 - sum([X for X in df[S] if not np.isnan(X)])
complete_ratios(x)
display(n)
display(x)

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
Species,,,,,,,,,,
A,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN
B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
C,NaN,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN
D,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0
total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
Species,,,,,,,,,,
A,0.264,0.578,NaN,0.0,0.905,NaN,NaN,0.000,0.0,0.108
B,NaN,0.422,NaN,NaN,0.095,NaN,NaN,0.268,0.0,0.465
C,0.212,0.000,0.0,NaN,0.000,0.0,0.0,0.732,NaN,0.427
D,NaN,0.000,0.0,NaN,0.000,0.0,0.0,0.000,NaN,0.000


In [6]:
def individual_from_total_and_ratio(a_df, r_df):
    for S in a_df:
        if not np.isnan(a_df[S].total):
            for i in r_df.index:
                if not np.isnan(r_df[S][i]) and np.isnan(a_df[S][i]):
                    a_df[S][i] = r_df[S][i]*a_df[S].total
individual_from_total_and_ratio(n,x)
display(n)
display(x)

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
Species,,,,,,,,,,
A,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,108.0
B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,465.0
C,NaN,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,427.0
D,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0
total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
Species,,,,,,,,,,
A,0.264,0.578,NaN,0.0,0.905,NaN,NaN,0.000,0.0,0.108
B,NaN,0.422,NaN,NaN,0.095,NaN,NaN,0.268,0.0,0.465
C,0.212,0.000,0.0,NaN,0.000,0.0,0.0,0.732,NaN,0.427
D,NaN,0.000,0.0,NaN,0.000,0.0,0.0,0.000,NaN,0.000


### Solve for Mixing Point (S7, S8, S10)

In [7]:
def total_from_individual_and_ratio(a_df, r_df):

n.S8.C = n.S10.C
n.S8.total = n.S8.C/x.S8.C
    
individual_from_total_and_ratio(n,x)
n.S7.A = n.S10.A
n.S7.B = n.S10.B-n.S8.B
totals_from_components(n)
individual_from_total_and_ratio(n,x)
display(n)
display(x)

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
Species,,,,,,,,,,
A,NaN,NaN,NaN,0.0,NaN,NaN,108.000000,0.000000,0.0,108.0
B,NaN,NaN,NaN,NaN,NaN,NaN,308.666667,156.333333,0.0,465.0
C,NaN,0.0,0.0,NaN,0.0,0.0,0.000000,427.000000,NaN,427.0
D,NaN,0.0,0.0,NaN,0.0,0.0,0.000000,0.000000,NaN,0.0
total,NaN,NaN,NaN,NaN,NaN,NaN,416.666667,583.333333,NaN,1000.0


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
Species,,,,,,,,,,
A,0.264,0.578,NaN,0.0,0.905,NaN,NaN,0.000,0.0,0.108
B,NaN,0.422,NaN,NaN,0.095,NaN,NaN,0.268,0.0,0.465
C,0.212,0.000,0.0,NaN,0.000,0.0,0.0,0.732,NaN,0.427
D,NaN,0.000,0.0,NaN,0.000,0.0,0.0,0.000,NaN,0.000


### Solve For Splitter (S3, S6, S7)

In [8]:
x.S7.A = n.S7.A/n.S7.total
x.S6.A = x.S7.A
x.S3.A = x.S6.A
n.S6.total = 2*n.S7.total
n.S3.total = n.S7.total
complete_ratios(x)
individual_from_total_and_ratio(n,x)
display(n)
display(x)

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
Species,,,,,,,,,,
A,NaN,NaN,108.000000,0.0,NaN,216.000000,108.000000,0.000000,0.0,108.0
B,NaN,NaN,308.666667,NaN,NaN,617.333333,308.666667,156.333333,0.0,465.0
C,NaN,0.0,0.000000,NaN,0.0,0.000000,0.000000,427.000000,NaN,427.0
D,NaN,0.0,0.000000,NaN,0.0,0.000000,0.000000,0.000000,NaN,0.0
total,NaN,NaN,416.666667,NaN,NaN,833.333333,416.666667,583.333333,NaN,1000.0


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
Species,,,,,,,,,,
A,0.264,0.578,0.2592,0.0,0.905,0.2592,0.2592,0.000,0.0,0.108
B,NaN,0.422,0.7408,NaN,0.095,0.7408,0.7408,0.268,0.0,0.465
C,0.212,0.000,0.0000,NaN,0.000,0.0000,0.0000,0.732,NaN,0.427
D,NaN,0.000,0.0000,NaN,0.000,0.0000,0.0000,0.000,NaN,0.000


### Solve for Unit II (S2, S5, S6)

In [9]:
lhs = x[['S2','S5']]['A':'B']
lhs.S5 = lhs.S5.apply(lambda X: -1*X)
rhs = n.S6['A':'B']
display(lhs)
display(rhs)
solution = np.linalg.solve(lhs,rhs)
n.S2.total = solution[0]
n.S5.total = solution[1]
individual_from_total_and_ratio(n,x)
display(n)

,S2,S5
Species,,
A,0.578,-0.905
B,0.422,-0.095


Species
A    216.000000
B    617.333333
Name: S6, dtype: float64

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
Species,,,,,,,,,,
A,NaN,951.254842,108.000000,0.0,735.254842,216.000000,108.000000,0.000000,0.0,108.0
B,NaN,694.514781,308.666667,NaN,77.181448,617.333333,308.666667,156.333333,0.0,465.0
C,NaN,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,427.000000,NaN,427.0
D,NaN,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.0
total,NaN,1645.769623,416.666667,NaN,812.436290,833.333333,416.666667,583.333333,NaN,1000.0


### Solve for Unit I (S1, S2, S3, S4)

In [10]:
n.S1.A = n.S5.A + n.S10.A
n.S1.B = n.S5.B + n.S10.B
n.S1.total = n.S1.A/x.S1.A
x.S1.B = n.S1.B/n.S1.total
complete_ratios(x)
individual_from_total_and_ratio(n,x)
display(n)
display(x)

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
Species,,,,,,,,,,
A,843.254842,951.254842,108.000000,0.0,735.254842,216.000000,108.000000,0.000000,0.0,108.0
B,542.181448,694.514781,308.666667,NaN,77.181448,617.333333,308.666667,156.333333,0.0,465.0
C,677.159191,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,427.000000,NaN,427.0
D,1131.551648,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.0
total,3194.147129,1645.769623,416.666667,NaN,812.436290,833.333333,416.666667,583.333333,NaN,1000.0


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
Species,,,,,,,,,,
A,0.264000,0.578,0.2592,0.0,0.905,0.2592,0.2592,0.000,0.0,0.108
B,0.169742,0.422,0.7408,NaN,0.095,0.7408,0.7408,0.268,0.0,0.465
C,0.212000,0.000,0.0000,NaN,0.000,0.0000,0.0000,0.732,NaN,0.427
D,0.354258,0.000,0.0000,NaN,0.000,0.0000,0.0000,0.000,NaN,0.000


In [11]:
n.S4.B = n.S1.B + n.S3.B - n.S2.B
n.S4.C = n.S1.C
n.S4.D = n.S1.D
totals_from_components(n)
display(n)

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
Species,,,,,,,,,,
A,843.254842,951.254842,108.000000,0.000000,735.254842,216.000000,108.000000,0.000000,0.0,108.0
B,542.181448,694.514781,308.666667,156.333333,77.181448,617.333333,308.666667,156.333333,0.0,465.0
C,677.159191,0.000000,0.000000,677.159191,0.000000,0.000000,0.000000,427.000000,NaN,427.0
D,1131.551648,0.000000,0.000000,1131.551648,0.000000,0.000000,0.000000,0.000000,NaN,0.0
total,3194.147129,1645.769623,416.666667,1965.044173,812.436290,833.333333,416.666667,583.333333,NaN,1000.0


### Solve for Unit III (S4, S8, S9)

In [14]:
n.S9.C = n.S4.C - n.S8.C
n.S9.D = n.S4.D
totals_from_components(n)
display(n)
display(x)

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
Species,,,,,,,,,,
A,843.254842,951.254842,108.000000,0.000000,735.254842,216.000000,108.000000,0.000000,0.000000,108.0
B,542.181448,694.514781,308.666667,156.333333,77.181448,617.333333,308.666667,156.333333,0.000000,465.0
C,677.159191,0.000000,0.000000,677.159191,0.000000,0.000000,0.000000,427.000000,250.159191,427.0
D,1131.551648,0.000000,0.000000,1131.551648,0.000000,0.000000,0.000000,0.000000,1131.551648,0.0
total,3194.147129,1645.769623,416.666667,1965.044173,812.436290,833.333333,416.666667,583.333333,1381.710839,1000.0


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
Species,,,,,,,,,,
A,0.264000,0.578,0.2592,0.0,0.905,0.2592,0.2592,0.000,0.0,0.108
B,0.169742,0.422,0.7408,NaN,0.095,0.7408,0.7408,0.268,0.0,0.465
C,0.212000,0.000,0.0000,NaN,0.000,0.0000,0.0000,0.732,NaN,0.427
D,0.354258,0.000,0.0000,NaN,0.000,0.0000,0.0000,0.000,NaN,0.000


## Calculate Solution

In [13]:
n.S2.total/n.S4.total

0.8375229655246947